In [1]:
import tensorflow as tf
import pandas as pd
import sagemaker
import numpy as np
tf.__version__

#Reading in data in folder
df = pd.read_csv('seaworldData.csv')
df.head()

,RideId,ParkId,CreatedTimeStamp,PresentWaitTime,PresentStatus,PresentIsActive,OneWaitTime,OneStatus,OneIsActive,TwoWaitTime,...,TwentyIsActive,TwentyOneWaitTime,TwentyOneStatus,TwentyOneIsActive,TwentyTwoWaitTime,TwentyTwoStatus,TwentyTwoIsActive,TwentyThreeWaitTime,TwentyThreeStatus,TwentyThreeIsActive
0,138,8,2020-07-14 00:10:00,0,Closed,0,0,Closed,0,0,...,0,0,Closed,0,0,Closed,0,0,Closed,0
1,139,8,2020-07-14 00:10:00,0,Closed,0,0,Closed,0,0,...,0,0,Closed,0,0,Closed,0,0,Closed,0
2,140,8,2020-07-14 00:10:00,0,Closed,0,0,Closed,0,0,...,0,0,Closed,0,0,Closed,0,0,Closed,0
3,141,8,2020-07-14 00:10:00,0,Closed,0,0,Closed,0,0,...,0,0,Closed,0,0,Closed,0,0,Closed,0
4,143,8,2020-07-14 00:10:00,0,Closed,0,0,Closed,0,0,...,0,0,Closed,0,0,Closed,0,0,Closed,0


In [2]:
numRows = df.shape[0]

xPoints = []
yPoints = []
for i in range(0, numRows, 21):
    timeFrame = df.loc[i:i+20]
    times = list(timeFrame[timeFrame.columns[range(3, 74, 3)]].to_numpy().reshape(1, 21*24)[0])
    statuses = list(timeFrame[timeFrame.columns[range(5, 75, 3)]].to_numpy().reshape(1, 21*24)[0])
    row = times + statuses
    row = np.array(row).reshape(42, 24)
    xPoints.append(row[:,0:12])
    yPoints.append(row[:,12:24])

xPoints = np.array(xPoints)
yPoints = np.array(yPoints)
p = np.random.permutation(xPoints.shape[0])
xPoints = xPoints[p]
yPoints = yPoints[p]

In [3]:
#sample reshape
#test = np.array(dataPoints).reshape(2, 1104)[0]
#for i in range(0, 46):
#    print(test.reshape(46, 24)[i])


In [4]:
size = xPoints.shape[0]
maxTrain = int(size * 0.8)
print(maxTrain)

trainData = {
    "x": xPoints[0:maxTrain],
    "y": yPoints[0:maxTrain]
}
testData = {
    "x": xPoints[maxTrain:],
    "y": yPoints[maxTrain:]
}

import pickle
with open('seaworld_train.pkl', 'wb') as f:
    pickle.dump(trainData, f)
with open('seaworld_test.pkl', 'wb') as f:
    pickle.dump(testData, f)


68494


In [5]:
import boto3
sagemaker_session = sagemaker.Session()

#Uploading data to S3 bucket titled "tf-iris-data"
prefix = "seaworld-data"
training_input_path = sagemaker_session.upload_data('seaworld_train.pkl', bucket='sagemaker-park-data', key_prefix=prefix)
training_input_path
testing_input_path = sagemaker_session.upload_data('seaworld_test.pkl', bucket='sagemaker-park-data', key_prefix=prefix)


In [6]:
print(training_input_path)
print(testing_input_path)

s3://sagemaker-park-data/seaworld-data/seaworld_train.pkl
s3://sagemaker-park-data/seaworld-data/seaworld_test.pkl
